# Two-Dimensional Langevin Field-Theoretic Simulation

This script runs Langevin field-theoretic simulation (L-FTS) in two-dimensional space with an AB diblock copolymer system using the discrete chain model. Note that two-dimensional L-FTS has no physical meaning. This tutorial is for pedagogical purposes only.

* This example is based on the equations in `PolymerFieldTheory/IncompressibleModel.ipynb` and uses the `polymer_field_theory.py` module.

* L-FTS is a partial saddle-point approximation (PSPA) method that approximates the functional representation of the canonical partition function by evaluating the following equation:
\begin{align}
\mathcal{Z} \propto \int \{\mathcal{D}\Omega_i\} \exp(-\beta H[\{\Omega_i\},\{\Omega_j^*\}]),
\end{align}
where $\{\Omega_i\}$ is a set of real fields and $\{\mathcal{D}\Omega_i\}$ is a set of functional integrals over all real fields.

* $\{\Omega_j^*\}$ is a set of saddle-point solutions for imaginary fields:

\begin{align}
\left.\frac{\delta H[\{\Omega_i\},\{\Omega_j\}]}{\delta \Omega_j}\right|_{\Omega_j({\bf r})=\Omega_j^*({\bf r})}=0.
\end{align}

* In L-FTS, the real fields evolve according to Langevin dynamics:
\begin{align}
\frac{\partial }{\partial \tau}\Omega_i({\bf r}, \tau) &= -λ_i \frac{\delta H[\{\Omega_i\},\{\Omega_j^*\}]}{\delta \Omega_i({\bf r}, \tau)} + \eta({\bf r},\tau),  \ \ \ \ (\textrm{per monomer unit}) \\
\end{align}
with noise statistics $\left<\eta({\bf r},\tau)\right> = 0$, $\left<\eta({\bf r},\tau)\eta({\bf r'},\tau')\right> =2\lambda_i k_B T \delta({\bf r}-{\bf r}')\delta(t-t')$. 
<!-- For convenience, we choose $\lambda_i = \frac{\beta }{\rho_0}$. -->

* The above equation can be discretized with the volume of a cell $\Delta V=\Delta x\Delta y\Delta z$ and $\delta\tau$ (Euler method in $\tau$):
\begin{align}
\lambda_i &\rightarrow \frac{\beta}{\rho_0}\lambda_i \\
\Omega_i({\bf r}, \tau+\delta\tau) &= \Omega_i({\bf r}, \tau) -λ_i\frac{\beta}{\rho_0}\frac{\delta H[\{\Omega_i\},\{\Omega_j^*\}]}{\delta \Omega_i({\bf r}, \tau)} \delta\tau + \sigma_i\mathcal{N}_i({\bf r}, \tau),  \ \ \ \ (\textrm{per monomer unit}) \\
\end{align}
where $\mathcal{N}$ is Gaussian noise and $\sigma_i^2 = \lambda_i\frac{2\delta\tau}{\Delta V \rho_0}$.

* In the 'per chain' unit:
\begin{align}
N\Omega_i &\rightarrow \Omega_i, \\
\Omega_i({\bf r}, \tau+\delta\tau) &= \Omega_i({\bf r}, \tau) -λ_i\frac{\beta}{C/R_0^3}\frac{\delta H[\{\Omega_i\},\{\Omega_j^*\}]}{\delta \Omega_i({\bf r}, \tau)} \delta\tau N + \sigma_i\mathcal{N}_i({\bf r}, \tau),  \ \ \ \ (\textrm{per chain unit}) \\
\end{align}
where $\mathcal{N}$ is Gaussian noise and $\sigma_i^2 = \lambda_i\frac{2\delta\tau N^2}{\Delta V \rho_0}$.

* The invariant polymerization index $\bar{N} = b^6\rho_0^2 N$. Then, $\sigma_i^2$ can be rewritten as:
\begin{align}
\Delta V \rho_0 &= \frac{\Delta V}{R_0^3} N \sqrt{\bar{N}}, \\
\sigma_i^2 &= \lambda_i\frac{2\delta\tau N}{\frac{\Delta V}{R_0^3}\sqrt{\bar{N}}}.  \ \ \ \ (\textrm{per chain unit}) \\
\end{align}

##### Example 1) AB-type polymeric systems (Euler method)

\begin{align}
\Omega_-({\bf r}, \tau+\delta\tau) &= \Omega_-({\bf r}, \tau) + \left[\Phi_-({\bf r},\tau) + \frac{2}{\chi}\Omega_-({\bf r},\tau)\right] \delta\tau + \sigma \mathcal{N}_-({\bf r}, \tau),  \ \ \ \ (\textrm{per monomer unit}) \\
\end{align}

\begin{align}
\Omega_-({\bf r}, \tau+\delta\tau) &= \Omega_-({\bf r}, \tau) + \left[\Phi_-({\bf r},\tau) + \frac{2}{\chi N}\Omega_-({\bf r},\tau)\right] \delta\tau N + \sigma \mathcal{N}_-({\bf r}, \tau)  \ \ \ \ (\textrm{per chain unit})
\end{align}

References:
* [(2013) Monte Carlo Field-Theoretic Simulations for Melts of Symmetric Diblock Copolymer](https://doi.org/10.1021/ma401687j)
* [(2014) A multi-species exchange model for fully fluctuating polymer field theory simulations](https://aip.scitation.org/doi/pdf/10.1063/1.4900574)
* [(2019) Computationally Efficient Field-Theoretic Simulations for Block Copolymer Melts](https://doi.org/10.1021/acs.macromol.9b01904)

### 1. Setting simulation parameters and initialization

#### Static simulation parameters

| Parameter | Symbol | Value |
|:----------|:------:|------:|
| Reference chain length | $N_\text{Ref}$ | 100 |
| Contour step | $\Delta s$ | 1/100 |
| Reference length | $R_0$ | $b N_\text{Ref}^{1/2}$ |
| Box size X | $L_x$ | 1.6 $R_0$ |
| Box size Y | $L_y$ | 1.6 $R_0$ |
| Grid points X | $m_x$ | 16 |
| Grid points Y | $m_y$ | 16 |
| Segment length A | $b_A/b$ | 1.0 |
| Segment length B | $b_B/b$ | 1.0 |
| Interaction parameter | $\chi N$ | 18 |
| A-block fraction | $f$ | 0.5 |

#### Langevin dynamics parameters

| Parameter | Symbol | Value |
|:----------|:------:|------:|
| Invariant polymerization index | $\bar{N}$ | 10000 |
| Time step | $\delta\tau N$ | 1.0 |
| Number of steps | - | 200 |

In [ ]:
import os
os.environ["OMP_NUM_THREADS"] = "1"      # Single-threaded OpenMP
os.environ["OMP_NUM_THREADS"] = "1"      # Single-threaded FFT

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import polymerfts
from polymerfts import PropagatorSolver

# Simulation parameters
nx = [16,16]                       # grid number
lx = [1.6,1.6]                     # box size
stat_seg_lengths = {"A":1.0,       # statistical segment lengths
                    "B":1.0}        
ds = 0.01                          # contour step interval
chi_n = {"A,B":18}                 # bare chi interaction parameter
monomer_types = ["A", "B"]         # monomer types

# AB diblock copolymer
blocks = [["A", 0.5, 0, 1],   # monomer type, contour length, start node, end node
          ["B", 0.5, 1, 2]]

# Langevin dynamics parameters
nsteps = 200                      # number of Langevin steps
dt   = 1.0                       # Langevin step interval, delta tau*N
nbar = 10000                     # invariant polymerization index
sigma = np.sqrt(2*dt*np.prod(nx)/(np.prod(lx)*np.sqrt(nbar))) # Langevin noise amplitude

# Create PropagatorSolver
solver = PropagatorSolver(
    nx=nx,
    lx=lx,
    ds=ds,
    bond_lengths=stat_seg_lengths,
    bc=["periodic", "periodic", "periodic", "periodic"],  # periodic BC for 2D
    chain_model="discrete",
    method="pseudospectral",
    reduce_memory=False,
)

# Add AB diblock copolymer
solver.add_polymer(volume_fraction=1.0, blocks=blocks)

### 2. Initial potential fields

In [ ]:
w_A =  6*np.tile(np.sin(np.linspace(0, 2*np.pi, nx[0])), (nx[1], 1))   # sinusoidal potential field for A
w_B = -6*np.tile(np.sin(np.linspace(0, 2*np.pi, nx[0])), (nx[1], 1))   # sinusoidal potential field for B
w = {"A": np.reshape(w_A, np.prod(nx)), "B": np.reshape(w_B, np.prod(nx))}  # potential field dictionary

# Plot each image with the same vmin and vmax
vmin = min(w_A.min(), w_B.min())
vmax = max(w_A.max(), w_B.max())

fig, axes = plt.subplots(1, 2, figsize=(10, 4))
fig.suptitle("Potential Fields")
im = axes[0].imshow(w_A, extent=(0, lx[1], 0, lx[0]), origin='lower', cmap='viridis', vmin=vmin, vmax=vmax)
im = axes[1].imshow(w_B, extent=(0, lx[1], 0, lx[0]), origin='lower', cmap='viridis', vmin=vmin, vmax=vmax)
axes[0].set(title='$w_A$', xlabel='y', ylabel='x')
axes[1].set(title='$w_B$', xlabel='y', ylabel='x')

fig.subplots_adjust(right=0.92)
fig.colorbar(im, ax=axes.ravel().tolist(), shrink=0.8)
fig.show()

### 3. Run L-FTS

* The effective Hamiltonian $H$ is written as:
\begin{align}
\frac{\beta H}{CV/R_0^3} &= -\sum_p \frac{\bar{\phi}_p}{\alpha_p}\log{Q_p} +
\frac{1}{V}\int d{\bf r}\left[\frac{\Omega_-^2({\bf r})}{\chi N} -\Omega_+({\bf r}) + \frac{\chi N}{4}\right].  \ \ \ \ (\textrm{per chain unit})
\end{align}

* The real fields are updated according to Langevin dynamics: 
\begin{align}
\Omega_i({\bf r}, \tau+\delta\tau) &= \Omega_i({\bf r}, \tau) -λ_i\frac{\beta}{C/R_0^3}\frac{\delta H[\{\Omega_i\},\{\Omega_j^*\}]}{\delta \Omega_i({\bf r}, \tau)} \delta\tau N + \sigma_i\mathcal{N}({\bf r}, \tau),  \ \ \ \ (\textrm{per chain unit}) \\
\end{align}

* The imaginary fields are updated using the equations below: 
\begin{align}
\Omega_j({\bf r}, \tau+\delta\tau) &= \Omega_j({\bf r}, \tau) +λ_j\frac{\beta}{C/R_0^3}\frac{\delta H[\{\Omega_i\},\{\Omega_j\}]}{\delta \Omega_j({\bf r}, \tau)} \delta\tau N,  \ \ \ \ (\textrm{per chain unit}) \\
\end{align}

##### Example 1) AB-type polymeric systems 

* The exchange field:
\begin{align}
\Omega_-({\bf r}, \tau+\delta\tau) &= \Omega_-({\bf r}, \tau) + λ_-\left[\phi_A({\bf r}) - \phi_B({\bf r}) + \frac{2}{\chi N}\Omega_-({\bf r},\tau)\right] \delta\tau N + \sigma \mathcal{N}_\sigma({\bf r}, \tau),  \ \ \ \ (\textrm{per chain unit}) \\
\end{align}

* The pressure-like field:
\begin{align}
\Omega_+({\bf r}, \tau+\delta\tau) &= \Omega_+({\bf r}, \tau) + \lambda_+ \left[\phi_A({\bf r}) + \phi_B({\bf r}) - 1\right] \delta\tau N,  \ \ \ \ (\textrm{per chain unit}) \\
\end{align}

In [ ]:
def compute_concentrations(w):
    # For the given fields, compute the polymer statistics
    solver.compute_propagators(w)
    solver.compute_concentrations()

    # Compute total concentration for each monomer type
    phi = {}
    for monomer_type in monomer_types:
        phi[monomer_type] = solver.get_concentration(monomer_type)
    return phi

def find_saddle_point(omega):
    global mpt, solver, chi_n, tolerance, lambda_plus, lambda_minus

    # Assign large initial value for error
    error_level = 1e20

    max_iter = 500  # maximum number of iterations

    # Saddle point iteration begins here
    for saddle_iter in range(1, max_iter+1):
        
        # Convert auxiliary potential fields to monomer potential fields
        w=mpt.to_monomer_fields_dict(omega) # {"A":omega[0]+omega[1], "B":-omega[0]+omega[1]}

        # Compute total concentrations
        phi = compute_concentrations(w)

        # Compute functional derivatives of Hamiltonian w.r.t. imaginary fields
        h_deriv_plus = np.reshape(mpt.compute_func_deriv(omega, phi, [1]), solver.n_grid) # 1.0-(phi["A"]+phi["B"])

        # Calculate Hamiltonian
        total_partitions = [solver.get_partition_function(p) for p in range(solver._molecules.get_n_polymer_types())]
        hamiltonian = mpt.compute_hamiltonian(solver._molecules, omega, total_partitions, include_const_term=True)

        # Compute total error
        error_level = np.std(h_deriv_plus)

        # Print iteration # and error levels
        if(error_level < tolerance or saddle_iter == max_iter or hamiltonian is np.nan):
        
            # Check the mass conservation
            mass_error = np.mean(h_deriv_plus)
            print("%8d %12.3E " % (saddle_iter, mass_error), end=" [ ")
            for p in range(solver._molecules.get_n_polymer_types()):
                print("%13.7E " % (solver.get_partition_function(p)), end=" ")
            print("] %15.9f   [" % (hamiltonian), end="")
            print("%13.7E" % error_level, end=" ")
            print("]")

        # Conditions to end the iteration
        if error_level < tolerance:
            break

        # Update pressure-like field using the simple gradient descent
        omega[1,:] -= h_deriv_plus*dt*lambda_plus

        # Set mean of pressure field to zero
        omega[1,:] -= np.mean(omega[1,:])

    # return the computed concentrations, Hamiltonian, iteration count, and error level
    return phi, hamiltonian, saddle_iter, error_level

# Convergence tolerance
tolerance = 1.0e-4  

# Fields update rates
lambda_minus = 1.0  
lambda_plus  = 3.0

# Random seed
np.random.seed(0)

# The number of components
M = len(monomer_types)

# Polymer field theory for Multimonomer system
mpt = polymerfts.SymmetricPolymerTheory(monomer_types, chi_n, zeta_n=None)

# Compute concentrations
phi = compute_concentrations(w)

# Initialize auxiliary fields for Langevin dynamics
omega = mpt.to_aux_fields(w)

# Store phi_A for each Langevin step
phi_A_history = []

# Langevin iteration begins here
for langevin_step in range(1, nsteps+1):
    print("Langevin step: ", langevin_step)

    # Compute functional derivatives of Hamiltonian w.r.t. exchange field
    h_deriv_minus = np.reshape(mpt.compute_func_deriv(omega, phi, [0]), solver.n_grid) # phi["A"] - phi["B"] + 2*omega[0,:]/chi_n["A,B"]

    # Update w_minus using first-order Euler method
    normal_noise = np.random.normal(0.0, sigma, [solver.n_grid])
    omega[0,:] += -h_deriv_minus*dt*lambda_minus + normal_noise*np.sqrt(lambda_minus)

    # Find saddle point of the pressure field
    print("iterations, mass error, total partitions, Hamiltonian, incompressibility error")
    phi, hamiltonian, saddle_iter, error_level = find_saddle_point(omega)

    phi_A_history.append(phi["A"])

### 4. Display the $\phi_A$ and $\phi_B$

In [ ]:
w = mpt.to_monomer_fields_dict(omega)

solver.compute_propagators(w)
solver.compute_concentrations()

# Get the ensemble average concentration for each monomer type
phi_A = np.reshape(solver.get_concentration("A"), nx)
phi_B = np.reshape(solver.get_concentration("B"), nx)

# Plot each image with the same vmin and vmax
fig, axes = plt.subplots(1, 2, figsize=(10, 4))
fig.suptitle("Concentrations")
im = axes[0].imshow(phi_A, extent=(0, lx[1], 0, lx[0]), origin='lower', cmap='RdBu_r', vmin=0.0, vmax=1.0)
im = axes[1].imshow(phi_B, extent=(0, lx[1], 0, lx[0]), origin='lower', cmap='RdBu_r', vmin=0.0, vmax=1.0)
axes[0].set(title='$\phi_A$', xlabel='y', ylabel='x')
axes[1].set(title='$\phi_B$', xlabel='y', ylabel='x')

fig.subplots_adjust(right=0.92)
fig.colorbar(im, ax=axes.ravel().tolist(), shrink=0.8)
fig.show()

### 5. Display the evolution of $\phi_A$

In [ ]:
import matplotlib.animation as animation
from IPython.display import HTML

# Create an animation of the chain propagator
fig, ax = plt.subplots(figsize=(8, 4))
fig.suptitle('Concentration of A', fontsize=16)
im = ax.imshow(np.reshape(phi_A_history[0], nx), extent=(0, lx[1], 0, lx[0]), origin='lower', cmap='RdBu_r', vmin=0.0, vmax=1.0) 
fig.colorbar(im, ax=ax)
def update(frame):
    im.set_array(np.reshape(phi_A_history[frame], nx))
    return [im]
anim = animation.FuncAnimation(fig, update, frames=len(phi_A_history), interval=1, repeat=False)

# Close the static figure to prevent it from displaying
plt.close(fig)

# Display only the animation
HTML(anim.to_jshtml())